<a href="https://colab.research.google.com/github/pkboom/metacode-machine-learning/blob/master/House_price_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import pandas as pd
import numpy as np

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
%cd /content/drive/MyDrive/curinc/ensemble/

/content/drive/MyDrive/curinc/ensemble


# ML 모델 학습

지금까지 배웠던 모델들을 활용해서 이전의 `EDA` 파일에서 만들었던 정제된 데이터셋으로 학습 및 평가를 진행하면 됩니다.     
집값 예측 데이터셋은 회귀 문제이므로, 회귀 모델들을 사용하면 됩니다.     

Ensemble 자료에서 배웠던 모델들은 각각 `Classifier`에서 `Regressor`로 이름을 바꾸면, 회귀 모델이 되는 점 참고해주세요.

In [80]:
X = pd.read_csv('./data/preprocessed_X.csv')
y = pd.read_csv('./data/preprocessed_y.csv')

y_1d = y.values.reshape(-1)
print("X dataset size:", X.shape)
print("y dataset size:", y_1d.shape)

X dataset size: (1457, 6)
y dataset size: (1457,)


In [88]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y_1d, test_size=0.2) 
X_train.shape, y_train.shape

((1165, 6), (1165,))

원하는 모델 구현해서 평가 데이터셋에 대한 성능을 찍고 분석해보세요.    
이 때, 회귀 모델에 맞는 평가 지표를 `sklearn.metrics`에서 찾아서 사용해보세요.

In [89]:
from sklearn.linear_model import SGDRegressor
import matplotlib.pyplot as plt


In [90]:
alpha = 0.001
lr = 1e-7
epochs = 1000

model = SGDRegressor(penalty='l2',
                      alpha=alpha,
                      max_iter=epochs,
                      tol=1e-3, 
                      learning_rate='invscaling',
                      eta0=lr)

model.fit(X_train, y_train)

model.intercept_

array([3.02324554e-06])

In [91]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)
loss = mean_squared_error(y_pred, y_test)
print('Linear Regression model less: %.4f' % loss)


Linear Regression model less: 0.1053


In [93]:
from sklearn.svm import SVR

svr = SVR(kernel='rbf', gamma=5, C=5)

svr.fit(X_train, y_train)

SVR(C=5, gamma=5)

In [94]:
y_pred = svr.predict(X_test)
loss = mean_squared_error(y_pred, y_test)
print(loss)

0.11888955712224814


In [95]:
from lightgbm import LGBMRegressor

lightgbm = LGBMRegressor(objective='regression', 
                         num_leaves=4,
                         learning_rate=0.01, 
                         n_estimators=5000,
                         max_bin=200, 
                         bagging_fraction=0.75,
                         bagging_freq=5, 
                         bagging_seed=7,
                         feature_fraction=0.2,
                         feature_fraction_seed=7,
                         verbose=-1,
                         #min_data_in_leaf=2,
                         #min_sum_hessian_in_leaf=11
                        )

lightgbm.fit(X_train, y_train)

LGBMRegressor(bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.2, feature_fraction_seed=7, learning_rate=0.01,
              max_bin=200, n_estimators=5000, num_leaves=4,
              objective='regression', verbose=-1)

In [96]:
y_pred = lightgbm.predict(X_test)

loss = mean_squared_error(y_pred, y_test)
loss

0.027304173445232442

Q1. 어떤 데이터 전처리 과정을 진행했는지 정리해주세요.

손실 데이터셋 제거, 아웃라이어 제거, 필요없는 피처 제거하거나 새로운 변수 생성, 몇몇 변수들의 값 정규화 등의 전처리들을 적용함.
또한 SalePrice 또한 로그 함수를 취해 값을 변화시킴.

Q2. 데이터 전처리릍 통해 데이터의 샘플 개수와 피처 차원은 어떻게 바뀌었나요?

처음 데이터셋의 크기는 (1460, 80)의 크기 였음(SalePrice 타겟 피처 고려 없이).
전처리를 통해 데이터셋의 모양은 (1457, 220)으로 바뀜.
학습에 방해가 될 샘플 3개를 제거하고, 범주형 데이터를 처리하기 위해 dummy 피처로 변화시키는 과정에서 220개의 피처 개수로 증가함.

Q3. 어떤 모델을 사용해서 데이터를 학습했으며, 모델의 장점과 단점을 설명해주세요.

Ensemble 모델들 중 최신 모델인 LightGBM과 XGBoost 모델을 사용했고 꽤 좋은 성능을 얻음.
Boosting 알고리즘들로, 결정 트리 기반으로 구현되었음에도 꽤 좋은 성능들을 보여주는 알고리즘임. 또한, 학습 속도 개선과 ensemble의 효과를 높이고자 한 기술들이 추가된 모델들임.